<h2><center>Campus Energy Consumption</center></h2>
<h5><center><a href="https://github.com/Aarsh-Wankar/">Aarsh Wankar</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110003@iitgn.ac.in">23110003@iitgn.ac.in
<br><br>
<a style="text-decoration:none" href="https://github.com/jsmaskeen/">Jaskirat Singh Maskeen</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110146@iitgn.ac.in">23110146@iitgn.ac.in</a>
<br><br>
<a style="text-decoration:none" href="https://github.com/nishchaybhutoria/">Nishchay Bhutoria</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110222@iitgn.ac.in">23110222@iitgn.ac.in</a>
<br><br>
<a style="text-decoration:none" href="https://github.com/ridhamp4/">Ridham Patel</a>, IIT Gandhinagar, <a style="text-decoration:none" href="mailto:23110238@iitgn.ac.in">23110238@iitgn.ac.in</a>

</h5>

In [25]:
from IPython.display import HTML
import plotly.io as pio
import warnings
warnings.filterwarnings("ignore")
pio.renderers.default = "notebook_connected"
HTML('''<button type="button" class="btn btn-outline-danger"  onclick="codeToggle();">Toggle Code</button>''')


In [26]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import tarfile
import io
import requests
from functools import reduce
from raceplotly.plots import barplot
import statsmodels.api as sm
from scipy.stats import pearsonr
import plotly.subplots as sp
from scipy import stats
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.linear_model import LinearRegression

### Loading the data and cleanup

In [27]:
consumption_full = pd.read_csv("data/consumption.csv")
q1 = pd.read_csv("data/q1.csv")
q2 = pd.read_csv("data/q2.csv")
q3 = pd.read_csv("data/q3.csv")
q4 = pd.read_csv("data/q4.csv")
chiller_q1 = pd.read_csv("data/chiller_q1.csv")
chiller_q2 = pd.read_csv("data/chiller_q2.csv")
chiller_q3 = pd.read_csv("data/chiller_q3.csv")
chiller_q4 = pd.read_csv("data/chiller_q4.csv")
weather = pd.read_csv("data/weather.csv")
weather22 = pd.read_csv("data/weather22.csv")
longer_supply = pd.read_csv("data/supply.csv")
overall = pd.concat([longer_supply,consumption_full.iloc[:,1:]],axis=1)
loss22 = pd.read_csv("data/loss22.csv")

In [28]:
q5 = overall[overall['Year'] == 22].iloc[0:3]
q6 = overall[overall['Year'] == 22].iloc[3:6]
q7 = overall[overall['Year'] == 22].iloc[6:9]
q8 = overall[overall['Year'] == 22].iloc[9:12]
quarter_ls = [q1,q2,q3,q4]
quarter_ls22 = [q5,q6,q7,q8]
consumption_df =  consumption_full.copy()
supply_df =  longer_supply.copy()
weather21_df = weather.copy().set_index('Metric').T.reset_index().rename(columns={'index': 'Month'})
weather22_df = weather22.copy().set_index('Metric').T.reset_index().rename(columns={'index': 'Month'})

In [29]:
humidity = weather[weather['Metric']=="Humidity"]
temp = list(weather[weather['Metric']=="Avg Temp"].iloc[0])
sunny_day_data = list(weather[weather['Metric']=="avg sun hours"].iloc[0])
humidity22 = weather22[weather22['Metric']=="Humidity"]
temp22 = list(weather22[weather22['Metric']=="Avg Temp"].iloc[0])
sunny_day_data22 = list(weather22[weather22['Metric']=="avg sun hours"].iloc[0])

In [30]:
solar_21 = longer_supply[longer_supply['Year']==21]['Solar Power']
solar_22 = longer_supply[longer_supply['Year']==22]['Solar Power']
temp21 = temp
temp22 = temp22

In [31]:
consumption_df['Date'] = pd.to_datetime('01-' + consumption_df['Month'], format='%d-%b.%y')
supply_df['Date'] = pd.to_datetime('01-' + supply_df['Month'] + '-' + supply_df['Year'].astype(str), format='%d-%b-%y')


months_dict = {'January': 1, 'February': 2, 'March': 3, 'April': 4, 
               'May': 5, 'June': 6, 'July': 7, 'August': 8,
               'September': 9, 'October': 10, 'November': 11, 'December': 12}
weather21_df['Month_Num'] = weather21_df['Month'].map(months_dict)
weather22_df['Month_Num'] = weather22_df['Month'].map(months_dict)

weather21_df['Year'] = 2021
weather22_df['Year'] = 2022
weather21_df['Date'] = pd.to_datetime(weather21_df['Year'].astype(str) + '-' + 
                                     weather21_df['Month_Num'].astype(str) + '-01')
weather22_df['Date'] = pd.to_datetime(weather22_df['Year'].astype(str) + '-' + 
                                     weather22_df['Month_Num'].astype(str) + '-01')

weather_df = pd.concat([weather21_df, weather22_df])
merged_df = pd.merge(consumption_df, supply_df[['Date', 'Torrent Power', 'Solar Power', 'Total']], on='Date', how='left')
merged_df = pd.merge(merged_df, weather_df[['Date', 'Avg Temp', 'Precipitation', 'Humidity', 'Rainy days', 'avg sun hours']], on='Date', how='left')

merged_df['Solar Percentage'] = (merged_df['Solar Power'] / merged_df['Total_x']) * 100
# merged_df.to_clipboard()

### Exploratory Data Analysis

#### Montly Electricity Consumption by facility

- The Chiller Plant shows the most pronounced seasonal variation, with peaks during the hottest months (April to September) and significantly lower consumption during winter months
- The Academic Area shows relatively stable consumption with slight increases during semesters.


In [32]:
facilities = ['Hostels & Guest house', 'Academic Area', 'Chiller Plant', 'Street light', 
              'WTP STP WSC SPS CWPS', 'Research park', 'Sports Complex']

fig = go.Figure()
for facility in facilities:
    fig.add_trace(go.Scatter(x=merged_df['Date'], y=merged_df[facility], mode='lines+markers', name=facility))

fig.update_layout(
    title='Monthly Electricity Consumption by Facility',
    xaxis_title='Month',
    yaxis_title='Consumption (kWh)',
    legend_title='Facility',
    height=600
)

#### Electricity Supply Composition

- The supply data shows that while Torrent Power (grid electricity) provides the majority of the energy needs, solar power makes a consistent contribution, with its relative share varying by season.

- The total electricity supply follows a similar seasonal pattern to total consumption, with higher demand during summer months when cooling needs increase.

In [33]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=supply_df['Date'], y=supply_df['Torrent Power'],
    mode='lines',
    name='Torrent Power',
    stackgroup='one'
))
fig.add_trace(go.Scatter(
    x=supply_df['Date'], y=supply_df['Solar Power'],
    mode='lines',
    name='Solar Power',
    stackgroup='one'
))

fig.update_layout(
    title='Monthly Electricity Supply Composition',
    xaxis_title='Month',
    yaxis_title='Supply (kWh)',
    legend_title='Source',
    height=500
)


#### Cumilative Power Consumption

In [34]:
df_long = consumption_full.melt(id_vars="Month", 
                                var_name="Category", 
                                value_name="Consumption")
month_order = {"Jan": 1, "Feb": 2, "Mar": 3, "Apr": 4, 
               "May": 5, "Jun": 6, "Jul": 7, "Aug": 8, 
               "Sep": 9, "Oct": 10, "Nov": 11, "Dec": 12}
df_long["Month_Num"] = df_long["Month"].map(month_order)
df_long = df_long.sort_values("Month_Num")
df_long["Cumulative_Consumption"] = df_long.groupby("Category")["Consumption"].cumsum()
df_long = df_long[df_long['Category'] != 'Total']
fig = px.bar(
    df_long,
    x="Cumulative_Consumption",
    y="Category",
    animation_frame="Month",
    orientation="h",
    range_x=[0, df_long["Cumulative_Consumption"].max() * 1.1],
    title="Cumulative Power Consumption",
    labels={"Cumulative_Consumption": "Cumulative Consumption (Units)", "Category": "Category"}
)
fig.update_layout(yaxis={'categoryorder': 'total ascending'})
fig.show()

### Primary Hypothesis

**IIT Gandhinagar's campus uses the highest supply of energy from Torrent
Power in August-October, and solar power is the best cure to address this
surge.**


In [35]:
data = {
    'Month': ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
              'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'] * 2,
    'Year': [21] * 12 + [22] * 12,
    'Torrent Power': list(longer_supply['Torrent Power'])
}
df = pd.DataFrame(data)
fig = px.line(
    df,
    x='Month',
    y='Torrent Power',
    color='Year',  
    title="Torrent Power Consumption Across Months (2021 vs. 2022)",
    labels={'Torrent Power': 'Energy Consumption (kWh)', 'Month': 'Month'},
    height=500
)

fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Energy Consumption (kWh)",
    legend_title="Year",
    template="plotly_white"
)

fig.show()

In [36]:
solar_data = list(map(lambda x:list(x['Solar Power'][:3]),quarter_ls+quarter_ls22))
solar_data = np.array(solar_data).flatten().tolist()
for i in range(len(solar_data)):
    solar_data[i]="".join(solar_data[i].split(","))
    solar_data[i] = float(solar_data[i])

solar_generation_data = {
    'Month': ['Jan 21','Feb 21','Mar 21','Apr 21','May 21','Jun 21','Jul 21','Aug 21','Sep 21','Oct 21','Nov 21','Dec 21','Jan 22','Feb 22','Mar 22','Apr 22','May 22','Jun 22','Jul 22','Aug 22','Sep 22','Oct 22','Nov 22','Dec 22'],
    'Generation': solar_data,
}

chiller_data = {
    'Month': ['Jan 21','Feb 21','Mar 21','Apr 21','May 21','Jun 21','Jul 21','Aug 21','Sep 21','Oct 21','Nov 21','Dec 21','Jan 22','Feb 22','Mar 22','Apr 22','May 22','Jun 22','Jul 22','Aug 22','Sep 22','Oct 22','Nov 22','Dec 22'],
    'Consumption': np.array([i['Chiller Plant'][0:3].map(lambda x:int(x.replace(',',''))).tolist() for i in quarter_ls] + [i['Chiller Plant'].tolist() for i in quarter_ls22]).flatten().tolist()
}

In [37]:
combined_data = {
    'Month': chiller_data['Month'],
    'Solar': solar_data,
    'Chiller': chiller_data['Consumption']
} 
dddf = pd.DataFrame(combined_data)
fig = px.line(dddf, x='Month', y=['Solar','Chiller'],
             title="Solar Generation vs Chiller Demand",
             labels={'value':'kWh','variable':'Source'})
fig.update_layout(yaxis2=dict(title='Chiller (kWh)', overlaying='y', side='right'))
fig.show()


### Sub-Hypotheses


#### 1. October Experiences Maximum Transmission Loss
<p>
  Transmission loss increases tremendously in October as the campus grid
  is heavily loaded by different activities. Though the chiller power
  consumption slightly drops due to favorable weather, this savings is
  overpowered by the increased demand from academic and sports facilities.
</p>

<p>
Q3 2022 is an outlier as there is increased usage of sports facilities due to InterIIT preparation.
</p>

In [38]:
temperatures = temp[1:] + temp22[1:]
q1_t = np.mean(temperatures[0:3])
q2_t = np.mean(temperatures[3:6])
q3_t = np.mean(temperatures[6:9])
q4_t = np.mean(temperatures[9:12])
q1_22_t = np.mean(temperatures[12:15])
q2_22_t = np.mean(temperatures[18:21])
q3_22_t = np.mean(temperatures[15:18])
q4_22_t = np.mean(temperatures[21:24])

In [39]:
loss_data = {
    'Quarter': ['Q1.21','Q2.21','Q3.21','Q4.21','Q1.22','Q2.22','Q3.22','Q4.22'],
    'Loss%': list(map(lambda x:float(x.iloc[4]['Total.1'].replace('%','')),quarter_ls)) + loss22.iloc[0,:].to_list(),
    'Temperature': [q1_t,q2_t,q3_t,q4_t,q1_22_t,q2_22_t,q3_22_t,q4_22_t]
}

fig = px.bar(loss_data, x='Quarter', y='Loss%',
            title="Transmission Losses by Quarter",
            color='Temperature')
fig.update_traces(texttemplate='%{y}%', textposition='outside')


#### 2. Increased Academic Block Power Usage in October
<p>
  During October, the academic calendar peaks with exams and related
  activities, pushing the energy consumption in academic buildings to
  higher levels. Key contributing factors include:
</p>
<ul>
  <li>Extended operating hours for classrooms, libraries, and laboratories.</li>
  <li>
    Enhanced use of air-conditioning, lighting, and electronic devices during
    exam times.
  </li>
  <li>Additional support for multimedia setups, projectors, and computers.</li>
</ul>


In [40]:
def make_convertible(s):
    if isinstance(s,int) or isinstance(s,float):
        return s
    s = s.split(",")
    s = "".join(s)
    return s
categories = ['Q1.21','Q2.21','Q3.21','Q4.21','Q1.22','Q2.22','Q3.22','Q4.22']
academic = list(map(lambda x:int(make_convertible(x.iloc[3]['Academic Area'])) , quarter_ls)) + list(map(lambda x:x.sum()['Academic Area'] , quarter_ls22))
hostel = list(map(lambda x:int(make_convertible(x.iloc[3]['Hostels & Guest house'])) , quarter_ls))+ list(map(lambda x:x.sum()['Hostels & Guest house'] , quarter_ls22))
sports = list(map(lambda x:int(make_convertible(x.iloc[3]['Sports Complex'])) , quarter_ls))+ list(map(lambda x:x.sum()['Sports Complex'] , quarter_ls22))

fig = go.Figure(data=[
    go.Bar(name='Academic Area', x=categories, y=academic),
    go.Bar(name='Hostel and Guest House', x=categories, y=hostel),
    go.Bar(name='Sports Complex', x=categories, y=sports)
])
fig.update_layout(barmode='stack', title="Energy Use: Academic vs Hostel and Guest House vs Sports Complex",yaxis_title='Energy Consumption (kWh)',xaxis_title='Quarter')


#### 3. Increased Power Usage in the Sports Complex During Q4
<p>
  In preparation for the Inter-IIT Sports Meet, the sports complex faces a
  spike in energy consumption driven by:
</p>
<ul>
  <li>High usage of floodlights during evening practice sessions.</li>
  <li>Extended operation of gym and training facilities.</li>
  <li>Increased demand for water heating in locker rooms and related amenities.</li>
</ul>

In [41]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=categories,
    y=academic,
    mode='lines+markers',
    name='Academic Area',
    line=dict(color='blue'),
    marker=dict(size=8)
))
fig.add_trace(go.Scatter(
    x=categories,
    y=hostel,
    mode='lines+markers',
    name='Hostels & Guest House',
    line=dict(color='red'),
    marker=dict(size=8),
    
))
fig.add_trace(go.Scatter(
    x=categories,
    y=sports,
    mode='lines+markers',
    name='Sports Complex',
    line=dict(color='green'),
    marker=dict(size=8),
    
))
fig.update_layout(
    title="Comparison of Energy Consumption: Academic vs Hostels",
    xaxis_title="Quarter",
    yaxis_title="Energy Consumption (kWh)",
    legend_title="Area",
    template="plotly_white",
    
)
fig.show()

#### 4. Reduced Chiller Power Consumption in October-November
<h5>Hypothesis: Temperature Significantly Influences Chiller Plant Consumption</h5> 

We expect higher temperatures to lead to increased cooling demand and therefore higher electricity consumption in the chiller plant.
<p>
  The chiller systems show a decline in energy consumption in October-November
  because of:
</p>
<ul>
  <li>Moderate average temperatures (approximately 27.9°C).</li>
  <li>Lower humidity levels (around 52%) compared to the monsoon peaks.</li>
  <li>
    Fewer rainy days, which reduce the latent heat load typically managed by
    chillers.
  </li>
</ul>



In [42]:
fig = px.area(chiller_data, x='Month', y='Consumption',
             title="Chiller Plant Energy Consumption",
             labels={'Consumption':'kWh'})
fig.add_scatter(x=chiller_data['Month'], y=chiller_data['Consumption'], 
               mode='lines+markers', name='Actual')
fig.show()


  Despite this reduction, the combined rise in power demand from academic
  and sports activities leads to an overall increase in campus energy need,
  resulting in the highest supply requirements from Torrent Power during
  October.

In [43]:
fig = px.scatter(merged_df, x='Avg Temp', y='Chiller Plant', 
                 trendline='ols', 
                 color=merged_df['Date'].dt.year,
                 labels={'Avg Temp': 'Average Temperature (°C)', 'Chiller Plant': 'Chiller Plant Consumption (kWh)'},
                 title='Temperature vs. Chiller Plant Consumption')
# Correlation analysis
correlation = stats.pearsonr(merged_df['Avg Temp'], merged_df['Chiller Plant'])
print(f"Pearson correlation: {correlation[0]}, p-value: {correlation[1]}")

# Linear regression model
X = sm.add_constant(merged_df['Avg Temp'])
model = sm.OLS(merged_df['Chiller Plant'], X).fit()
print(model.summary())
fig.show()

Pearson correlation: 0.905512621751208, p-value: 1.1864479802036517e-09
                            OLS Regression Results                            
Dep. Variable:          Chiller Plant   R-squared:                       0.820
Model:                            OLS   Adj. R-squared:                  0.812
Method:                 Least Squares   F-statistic:                     100.2
Date:                Fri, 18 Apr 2025   Prob (F-statistic):           1.19e-09
Time:                        19:12:57   Log-Likelihood:                -290.40
No. Observations:                  24   AIC:                             584.8
Df Residuals:                      22   BIC:                             587.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

Results:

- The Pearson correlation coefficient between average temperature and chiller plant consumption is 0.90 with a p-value < 0.001, indicating a strong, statistically significant positive correlation

- The linear regression model shows that for each 1°C increase in average temperature, chiller plant consumption increases by approximately 22,970 kWh per month, with temperature explaining about 82% of the variation in chiller plant consumption ($\text{R}^2$ = 0.82)

Conclusion:

- We can confirm the hypothesis that temperature significantly influences chiller plant consumption, with higher temperatures leading to substantially higher electricity consumption.
- This relationship is shows the need for energy planning during summer months when temperatures regularly exceed 30°C



#### 5. Why especially in February-March for both years, increasing solar power generation capabilities minimized external power dependency even more effectively?

In [44]:
fig = px.area(solar_generation_data, x='Month', y='Generation',
             title="Solar Power Generation (2021)",
             labels={'Generation':'kWh'})
fig.add_scatter(x=solar_generation_data['Month'], y=solar_generation_data['Generation'], 
               mode='lines+markers', name='Actual')
fig.show()


<p>
  The following table presents the monthly environmental metrics for IIT
  Gandhinagar, highlighting why February-March stands out:
</p>

<table>
  <caption>Monthly Environmental Metrics at IIT Gandhinagar for 2021</caption>
  <thead>
    <tr>
      <th>Metric</th>
      <th>January</th>
      <th>February</th>
      <th>March</th>
      <th>April</th>
      <th>May</th>
      <th>June</th>
      <th>July</th>
      <th>August</th>
      <th>September</th>
      <th>October</th>
      <th>November</th>
      <th>December</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Avg Temp (°C)</td>
      <td>20.4</td>
      <td>22.8</td>
      <td>27.2</td>
      <td>31.2</td>
      <td>33.0</td>
      <td>31.9</td>
      <td>28.5</td>
      <td>27.5</td>
      <td>28.0</td>
      <td>27.9</td>
      <td>25.0</td>
      <td>21.5</td>
    </tr>
    <tr>
      <td>Precipitation (mm)</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>1.0</td>
      <td>73.0</td>
      <td>307.0</td>
      <td>242.0</td>
      <td>109.0</td>
      <td>17.0</td>
      <td>3.0</td>
      <td>1.0</td>
    </tr>
    <tr>
      <td>Humidity (%)</td>
      <td>43.0</td>
      <td>39.0</td>
      <td>31.0</td>
      <td>31.0</td>
      <td>42.0</td>
      <td>58.0</td>
      <td>76.0</td>
      <td>80.0</td>
      <td>74.0</td>
      <td>52.0</td>
      <td>44.0</td>
      <td>46.0</td>
    </tr>
    <tr>
      <td>Rainy days</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>0</td>
      <td>5</td>
      <td>14</td>
      <td>15</td>
      <td>7</td>
      <td>2</td>
      <td>1</td>
      <td>0</td>
    </tr>
    <tr>
      <td>Avg Sun Hours</td>
      <td>9.7</td>
      <td>10.2</td>
      <td>10.8</td>
      <td>11.4</td>
      <td>11.3</td>
      <td>9.5</td>
      <td>7.1</td>
      <td>6.2</td>
      <td>8.0</td>
      <td>10.1</td>
      <td>9.9</td>
      <td>9.6</td>
    </tr>
  </tbody>
</table>
<br>
<table><caption>Monthly Environmental Metrics at IIT Gandhinagar for 2022</caption><thead><tr><th>Metric</th><th>January</th><th>February</th><th>March</th><th>April</th><th>May</th><th>June</th><th>July</th><th>August</th><th>September</th><th>October</th><th>November</th><th>December</th></tr></thead><tbody><tr><td>Avg Temp (°C)</td><td>19.7</td><td>23.3</td><td>27.3</td><td>31.8</td><td>34.0</td><td>33.2</td><td>28.4</td><td>28.4</td><td>29.1</td><td>29.4</td><td>24.7</td><td>22.4</td></tr><tr><td>Precipitation (mm)</td><td>1</td><td>1</td><td>1</td><td>2</td><td>7</td><td>80</td><td>290</td><td>265</td><td>85</td><td>12</td><td>2</td><td>1</td></tr><tr><td>Humidity (%)</td><td>50</td><td>42</td><td>33</td><td>34</td><td>42</td><td>59</td><td>77</td><td>80</td><td>74</td><td>56</td><td>50</td><td>50</td></tr><tr><td>Rainy Days</td><td>0</td><td>0</td><td>0</td><td>0</td><td>1</td><td>4</td><td>12</td><td>11</td><td>5</td><td>1</td><td>0</td><td>0</td></tr><tr><td>Avg Sun Hours</td><td>9.5</td><td>10.0</td><td>9.0</td><td>10.0</td><td>10.5</td><td>8.0</td><td>4.0</td><td>3.5</td><td>7.5</td><td>9.5</td><td>9.0</td><td>9.5</td></tr></tbody></table>


<h5>Optimal Conditions for Solar Efficiency in Q1</h5>
<p>
  Q1 is particularly ideal for solar power production because:
</p>
<ul>
  <li>
    The average temperature of about 25.4°C is moderate enough to minimize thermal
    losses, which can hamper solar panel efficiency in excessively hot months.
  </li>
  <li>
    With an average of 10 sun hours per day in February - March, there is ample
    sunlight to drive significant solar power generation.
  </li>
  <li>
    Low precipitation and minimal rainy days reduce cloud cover, ensuring more
    consistent solar exposure.
  </li>
</ul>


In [45]:
plot_data = pd.DataFrame({
    'Month': ['Jan 21','Feb 21','Mar 21','Apr 21','May 21','Jun 21','Jul 21','Aug 21','Sep 21','Oct 21','Nov 21','Dec 21',
              'Jan 22','Feb 22','Mar 22','Apr 22','May 22','Jun 22','Jul 22','Aug 22','Sep 22','Oct 22','Nov 22','Dec 22'],
    'Sunny Days': sunny_day_data[1:] + sunny_day_data22[1:],
    'Avg Temp (°C)': temp[1:] + temp22[1:],
    'Solar Generation (kWh)': list(np.array(solar_data))
})

# Extract month name for matching
plot_data['MonthOnly'] = [m.split()[0] for m in plot_data['Month']]

# Scatter plot
fig = px.scatter(
    plot_data,
    x='Avg Temp (°C)',
    y='Sunny Days',
    size='Solar Generation (kWh)',
    color='Solar Generation (kWh)',
    hover_name='Month',
    size_max=40,
    color_continuous_scale='YlOrRd',
    title='Solar Generation vs Sunny Days vs Temperature',
    labels={
        'Sunny Days': 'Sunny Days (per month)',
        'Avg Temp (°C)': 'Average Temperature (°C)',
        'Solar Generation (kWh)': 'Solar Generation (kWh)'
    }
)

# Add lines between same months
for month in plot_data['MonthOnly'].unique():
    same_months = plot_data[plot_data['MonthOnly'] == month]
    if len(same_months) == 2:
        fig.add_trace(go.Scatter(
            x=same_months['Avg Temp (°C)'],
            y=same_months['Sunny Days'],
            mode='lines',
            line=dict(color='gray', dash='dot'),
            showlegend=False,
            hoverinfo='skip'
        ))

# Final layout tweaks
fig.update_layout(template='plotly_white')

fig.show()

##### Related Hypothesis: Monsoon Season Negatively Impacts Solar Power Generation

We examine, whether the monsoon season (July-September), characterized by higher precipitation and cloud cover, significantly reduces solar power generation compared to other seasons.


In [46]:
merged_df['Monsoon'] = merged_df['Date'].dt.month.isin([7, 8, 9])

fig = px.box(merged_df, x='Monsoon', y='Solar Power', 
             labels={'Monsoon': 'Monsoon Season', 'Solar Power': 'Solar Power Generation (kWh)'},
             title='Solar Power Generation: Monsoon vs. Non-Monsoon Months')

monsoon_solar = merged_df[merged_df['Monsoon']]['Solar Power']
non_monsoon_solar = merged_df[~merged_df['Monsoon']]['Solar Power']
t_test = stats.ttest_ind(monsoon_solar, non_monsoon_solar, equal_var=False)
print(f"T-test: t-statistic = {t_test[0]}, p-value = {t_test[1]}")

for weather_metric in ['Precipitation', 'avg sun hours']:
    corr = stats.pearsonr(merged_df[weather_metric], merged_df['Solar Power'])
    print(f"Correlation with {weather_metric}: {corr[0]}, p-value: {corr[1]}")

fig.show()

T-test: t-statistic = -4.884988385694464, p-value = 0.000141518221506433
Correlation with Precipitation: -0.5737327832890389, p-value: 0.003375382138642519
Correlation with avg sun hours: 0.5863326032773123, p-value: 0.0026025219214477176


Results:
- The t-test comparing solar power generation between monsoon and non-monsoon months yields a t-statistic of -4.88 with a p-value of 0.0001, confirming a statistically significant difference. 
- During monsoon months, solar power generation decreases by approximately 24% compared to non-monsoon months. 
- The correlation between solar power and precipitation is -0.57 (p = 0.003), while the correlation with average sun hours is 0.58 (p = 0.002).

Conclusion: 
We can confirm the hypothesis that monsoon season negatively impacts solar power generation. The reduction is primarily due to decreased sunlight hours and increased cloud cover during this period. 

This suggests the need for alternative renewable power sources during monsoon months to compensate for reduced solar output.